In [ ]:
# !pip3 install openpyxl

In [46]:
import requests
import base64
import os
import numpy as np
import pandas as pd
import json
import time
pd.set_option('display.max_columns', 500)

In [26]:
class Spotify():
    '''
    Class to handle hitting spotify API to get music features
    '''
    
    def __init__(self, 
                 client_id=None, 
                 client_secret=None):
        
        self.client_id = client_id
        self.client_secret = client_secret
        
        self.get_access_token()
        
        
    def get_access_token(self):
        ## Example use:
        ## spotify_instance = Spotify(CLIENT_ID, CLIENT_SECRET)
        ## spotify_instance.get_access_token()
        
        secret_bytes = bytes(('{}:{}'.format(self.client_id, self.client_secret)),'utf-8')
        secret_enc = base64.b64encode(secret_bytes).decode('utf-8')
        
        data = {'grant_type': 'client_credentials'}
        headers = {'Authorization': 'Basic {}'.format(secret_enc)}
        url = 'https://accounts.spotify.com/api/token'
        r = requests.post(url, headers=headers, data=data)
        
        self.access_token = r.json()['access_token']
    
    def lookup_spotify_id(self, id_list):
        '''
        this takes in ISRC and returns ISRC to Spotify ID map
        '''
        df = pd.DataFrame([])
        bad_ids = []
        for i in id_list:
            r = requests.get(f'https://api.spotify.com/v1/search?type=track&q=isrc:{i}',
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                print('sleeping for {}'.format(retry_secs))
                time.sleep(retry_secs)
            try:
                spotify_id = r.json()['tracks']['items'][0]['id']
                row = {'isrc': str(i), 'spotify_id': spotify_id}
                df = df.append(row, ignore_index = True)
            except:
#                 print(f'id issue {len(bad_ids) + 1}')
                row = {'isrc': str(i), 'spotify_id': None}
                df = df.append(row, ignore_index = True)
                bad_ids.append(i)
        return df
    
    def lookup_album_id(self, id_list):
        '''
        this takes in UPC and returns UPC to Spotify Album ID
        '''
        df = pd.DataFrame([])
        bad_ids = []
        for i in id_list:
            r = requests.get(f'https://api.spotify.com/v1/search?type=album&q=upc:{i}',
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                print('sleeping for {}'.format(retry_secs))
                time.sleep(retry_secs)
            try:
                spotify_id = r.json()['albums']['items'][0]['id']
                row = {'upc': str(i), 'spotify_album_id': spotify_id}
                df = df.append(row, ignore_index = True)
            except:
#                 print(f'id issue {len(bad_ids) + 1}')
                row = {'upc': str(i), 'spotify_album_id': None}
                df = df.append(row, ignore_index = True)
                bad_ids.append(i)
        return df
    
    def group_ids(self, list_of_ids, bucket_size=100):
        list_of_ids = pd.DataFrame(list_of_ids, columns=['id'])
        list_of_ids['bucket'] = np.floor(np.arange(len(list_of_ids)) / bucket_size)
        
        gpd_ids = list_of_ids.groupby('bucket')['id'].apply(lambda x: ','.join(x))
        
        return gpd_ids

    def get_audio_features(self, id_list, bucket_size=100):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['acousticness', 'liveness', 'instrumentalness', 
                    'analysis_url', 'uri', 'time_signature', 'loudness', 
                    'speechiness', 'duration_ms', 'danceability', 'mode', 
                    'id', 'energy', 'key', 'track_href', 
                    'valence', 'type', 'tempo']
        
        df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/audio-features?ids={}'.format(x), 
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
            
            audio_features = r.json()['audio_features']
            audio_features = [i for i in audio_features if i]
            
            for row in audio_features:
                for c in col_list:
                    if c not in row:
                        row[c] = None
                df.append(row)
    
        return pd.DataFrame(df)
    
    def get_tracks(self, id_list, bucket_size=50):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        
        track_df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/tracks?ids={}'.format(x),
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
                  
            tracks = r.json()['tracks']
            tracks = [i for i in tracks if i]
            
            for t in tracks:
                df_row = {'id':t['id'],
                          'popularity': t['popularity'],
                          'release_date': t['album']['release_date'],
                          'release_date_precision': t['album']['release_date_precision']}
                track_df.append(df_row)
        
        final_track_df = pd.DataFrame(track_df)        
        return final_track_df

    def get_albums(self, id_list, bucket_size=20):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['id','release_date', 'release_date_precision','total_tracks','type']
        
        album_df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/albums?ids={}'.format(x),
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
                  
            albums = r.json()['albums']
            albums = [i for i in albums if i]
            
            for a in albums:
                df_row = {x: a[x] for x in col_list}
                album_df.append(df_row)
                
        final_album_df = pd.DataFrame(album_df)
        return final_album_df

In [27]:
client_id = 'ae17fb14354d4d98a442007563fafab9'
client_secret = 'cb9e3752f07f4cfc9685c9369fd4b11c'

sp = Spotify(client_id,client_secret)
sp.get_access_token()

In [31]:
data_path = r'data/'
dataset_path = r'data/mri_full_data.csv'

music_data = pd.read_csv(dataset_path, low_memory=False)
music_data.drop('Unnamed: 0', axis=1, inplace=True)
music_data['Display Upc'] = music_data['Display Upc'].astype('string')

In [32]:
isrc_ids = list(music_data[~music_data.ISRC.isna()].ISRC.unique())
upc_ids = list(music_data[~music_data['Display Upc'].isna()]['Display Upc'].unique())

In [33]:
isrc_to_spotify_id = sp.lookup_spotify_id(isrc_ids)
upc_to_spotify_album_id = sp.lookup_album_id(upc_ids)

isrc_to_spotify_id.to_csv(os.path.join(data_path, 'isrc_to_spotify_id.csv'))
upc_to_spotify_album_id.to_csv(os.path.join(data_path, 'upc_to_spotify_album_id.csv'))

In [34]:
spotify_ids = (isrc_to_spotify_id[~(isrc_to_spotify_id.spotify_id.isna())].spotify_id.unique())

track_audio_feats = sp.get_audio_features(spotify_ids)
track_audio_feats.to_csv(os.path.join(data_path, 'track_audio_feats.csv'))

In [35]:
track_information = sp.get_tracks(spotify_ids)
track_information.to_csv(os.path.join(data_path, 'track_information.csv'))

In [36]:
spotify_album_ids = list(
    upc_to_spotify_album_id[
        ~(upc_to_spotify_album_id.spotify_album_id.isna())
    ].spotify_album_id.unique()
)

album_information = sp.get_albums(spotify_album_ids)
album_information.to_csv(os.path.join(data_path, 'album_information.csv'))

# join & write all data

In [37]:
track_information.head()

,id,popularity,release_date,release_date_precision
0,4TyCnstYu6LGrjka5WW6ft,58,2014-06-10,day
1,49CdYBpfABUa0ZfT8FizQZ,55,2014-06-10,day
2,0dE7c9rhBuEahVmYVYuooI,49,2021-03-26,day
3,0BweE3lWBMXRPWWLtLV5z8,57,2014-06-10,day
4,0y6EUHrKgUIEc0yqGg4acO,37,2018-06-15,day


In [49]:
music_data_merged_spotify = music_data.merge(
    isrc_to_spotify_id,
    how = 'left',
    left_on = 'ISRC',
    right_on = 'isrc'
).merge(
    upc_to_spotify_album_id,
    how = 'left',
    left_on = 'Display Upc',
    right_on = 'upc'
).merge(
    track_audio_feats,
    how = 'left',
    left_on = 'spotify_id',
    right_on = 'id'
).drop(
    ['id', 'uri','track_href','analysis_url','type'],
    axis = 1
).merge(
    album_information,
    how = 'left',
    left_on = 'spotify_album_id',
    right_on = 'id'
).rename(
    columns = {'release_date': 'album_release_date',
               'release_date_precision': 'album_release_date_precision',
               'total_tracks': 'album_total_tracks'}
).drop(
    ['id', 'isrc', 'upc'],
    axis = 1
).merge(
    track_information,
    how = 'left',
    left_on = 'spotify_id',
    right_on = 'id'
).rename(
    columns = {'release_date': 'track_release_date',
               'release_date_precision': 'track_release_date_precision',
               'popularity': 'track_popularity'}
).drop(
    'id', axis = 1
)

In [50]:
music_data_merged_spotify['type'] = np.where(
    (
        (music_data_merged_spotify.type.isna()) &
        ~(music_data_merged_spotify.spotify_id.isna())
    ) |
    (
        ~(music_data_merged_spotify.spotify_id.isna()) &
        ~(music_data_merged_spotify.spotify_album_id.isna())
    ),
    'track',
    music_data_merged_spotify.type
)

In [51]:
music_data_merged_spotify.to_csv(os.path.join(data_path, 'music_data_merged_spotify.csv'))

# joining stats

In [52]:
len_match = len(music_data_merged_spotify[~(music_data_merged_spotify.type.isna())])
len_df = len(music_data_merged_spotify)
print (f'{len_match} out of {len_df} rows have spotify information ({np.round((len_match/len_df) * 100, 1)}%)')

4704716 out of 4757930 rows have spotify information (98.9%)


In [55]:
music_data_merged_spotify[
    (music_data_merged_spotify.album_release_date != music_data_merged_spotify.track_release_date) &
    ~(music_data_merged_spotify.album_release_date.isna()) &
    ~(music_data_merged_spotify.track_release_date.isna())
]

,Statement Month,Store Name,Label Name,Artist Name,Release Name,Track Artist (Performer),Track Name,ISRC,Display Upc,Country Code,Continent,"Stream Ad-Supp, Stream Prem, Download, Physical, Locker, Other",Units,Gross Revenue USD,Net Revenue USD,spotify_id,spotify_album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,album_release_date,album_release_date_precision,album_total_tracks,type,track_popularity,track_release_date,track_release_date_precision
61,2018-07,iTunes/Apple,MRI Entertainment,Rebelution,Free Rein,Rebelution,Celebrate,US4CL1810001,657481106983,US,North America,Stream Premium,"302,307","$2,251.00","$2,025.90",26IEVYmeFletNA10cOkdBb,5FfbqVo0OapeCvoJvjWlOp,0.660,0.679,2.0,-6.749,1.0,0.1110,0.02240,0.000000,0.0945,0.595,164.110,243174.0,4.0,2018-06-15,day,12.0,track,49.0,2018-02-23,day
78,2018-07,Spotify,MRI Entertainment,Rebelution,Free Rein,Rebelution,Celebrate,US4CL1810001,657481106983,US,North America,Stream Premium,"439,003","$2,079.43","$1,871.49",26IEVYmeFletNA10cOkdBb,5FfbqVo0OapeCvoJvjWlOp,0.660,0.679,2.0,-6.749,1.0,0.1110,0.02240,0.000000,0.0945,0.595,164.110,243174.0,4.0,2018-06-15,day,12.0,track,49.0,2018-02-23,day
83,2018-08,iTunes/Apple,MRI Entertainment,Rebelution,Free Rein,Rebelution,Celebrate,US4CL1810001,657481106983,US,North America,Stream Premium,"271,151","$2,055.10","$1,849.59",26IEVYmeFletNA10cOkdBb,5FfbqVo0OapeCvoJvjWlOp,0.660,0.679,2.0,-6.749,1.0,0.1110,0.02240,0.000000,0.0945,0.595,164.110,243174.0,4.0,2018-06-15,day,12.0,track,49.0,2018-02-23,day
114,2018-03,iTunes/Apple,MRI Entertainment,Rebelution,Free Rein,Rebelution,Celebrate,US4CL1810001,657481106983,US,North America,Download,"2,061","$1,875.51","$1,687.96",26IEVYmeFletNA10cOkdBb,5FfbqVo0OapeCvoJvjWlOp,0.660,0.679,2.0,-6.749,1.0,0.1110,0.02240,0.000000,0.0945,0.595,164.110,243174.0,4.0,2018-06-15,day,12.0,track,49.0,2018-02-23,day
140,2018-04,iTunes/Apple,MRI Entertainment,Rebelution,Free Rein,Rebelution,Celebrate,US4CL1810001,657481106983,US,North America,Stream Premium,"207,186","$1,723.26","$1,550.93",26IEVYmeFletNA10cOkdBb,5FfbqVo0OapeCvoJvjWlOp,0.660,0.679,2.0,-6.749,1.0,0.1110,0.02240,0.000000,0.0945,0.595,164.110,243174.0,4.0,2018-06-15,day,12.0,track,49.0,2018-02-23,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4757920,2020-08,iTunes/Apple,MRI Entertainment,Rebelution,Dub Collection,Rebelution,Attention Span Dub,US4CL2010024,657481108185,US,North America,Download,-122,-$111.02,-$99.92,0ijA161YVenpYKjl7ZH6MM,17DzOLiTDWTvdtxTE4TxzW,0.589,0.685,6.0,-5.274,0.0,0.0309,0.00846,0.673000,0.2870,0.562,147.216,345448.0,4.0,2020-07-17,day,15.0,track,42.0,2020-05-29,day
4757923,2021-07,iTunes/Apple,MRI Entertainment,Rebelution,In the Moment,Rebelution,Satisfied,US4CL2110024,657481109182,US,North America,Download,-343,-$312.13,-$280.92,0dE7c9rhBuEahVmYVYuooI,4ZO52CTQ2EzwDmAIrFjHaW,0.758,0.697,1.0,-6.839,1.0,0.0664,0.01840,0.000002,0.0680,0.776,146.910,196550.0,4.0,2021-06-18,day,15.0,track,49.0,2021-03-26,day
4757924,2021-07,iTunes/Apple,MRI Entertainment,Rebelution,In the Moment,Rebelution,Heavy as Lead,US4CL2110026,657481109182,US,North America,Download,-514,-$467.74,-$420.97,59HQVKZfaKycUzRTtp1zDO,4ZO52CTQ2EzwDmAIrFjHaW,0.675,0.806,0.0,-6.788,1.0,0.2100,0.05710,0.000000,0.0759,0.559,165.900,176762.0,4.0,2021-06-18,day,15.0,track,52.0,2021-05-21,day
4757925,2021-07,iTunes/Apple,MRI Entertainment,Rebelution,In the Moment,Rebelution,Old School Feeling,US4CL2110025,657481109182,US,North America,Download,-572,-$520.52,-$468.47,54GuCaNBEr3dcPAZq4SNkB,4ZO52CTQ2EzwDmAIrFjHaW,0.975,0.624,11.0,-6.891,0.0,0.0512,0.03730,0.002540,0.0831,0.965,125.049,236625.0,4.0,2021-06-18,day,15.0,track,47.0,2021-04-23,day
